# <center>BZAN 545 Final Overview</center>

***

## 1 - Class Topics:

***

### Class 21: March 31, 2021

1. Review Bonus 14
2. 2.7.2 - Read data from Postgres into Python
3. 3.4.3 - PostegreSQL: Conditional Processing
4. 3.5.3 - PostgreSQL: Timing Code
5. 3.7.3 - PostegreSQL: Creating Dummy Variables
6. 3.8.3 - PostegreSQL: Aggregating

### Class 22: April 5, 2021

1. Review Bonus 15
2. 3.9.3 - PostegreSQL: Merging
3. 3.11.3 – PostegreSQL: Working with Dates

### Class 23: April 7, 2021

1. Review Bonus 16
2. 3.13.3 - PostegreSQL: Creating Functions
3. 3.15.3 - PostegreSQL: Speeding Up Code

### Class 26: April 19, 2021

1. Review Bonus 17
2. 3.15.2.1 - Spark Components
3. 3.15.2.2 - Spark: How does it speed up data processing
4. 3.15.2.3 - Spark: Installaion 

### Class 27: April 21, 2021

1. Review Bonus 18
2. 3.15.2.4 - Spark: Main Operations

### Class 28: April 26, 2021

1. Review Bonus 19
2. 4 - MongoDB: PyMongo
3. Bonus 20

***
## 2 - Bonus Assignments
***

### Bonus 14 (Postgres)

**Topics**: Operators and Missing Values

***
#### Question 1 
***

Use the modulo operator to compute the remainder of 444 divded by 7.

**Answer**: `SELECT 444 % 7;`

**Output**: $3$

***    
#### **Question 2**
***

Compute the absolute value of $-9$ using an operator.

**Answer:** `SELECT @ -9;`

**Output:** $9$

***
#### Question 3
***

Consider the following data: 

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a (cola int, colb int);
INSERT INTO a VALUES (1,null),(2,3),(5,null);
SELECT * FROM a;
```

Return all the records that have a null in `colb`

**Answer:**`SELECT * FROM a WHERE colb IS NULL;`

**Or:**`SELECT * FROM a WHERE colb IS NOT DISTINCT FROM NULL;`

***
#### Question 4
***

Consider the following tables:

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a (cola int, colb int);
INSERT INTO a VALUES (1,null),(2,3),(5,null);
SELECT * FROM a;

DROP TABLE IF EXISTS b;
CREATE TABLE b (cola int, colb int);
INSERT INTO b VALUES (1,null),(1,1),(5,null);
SELECT * FROM b;
```

Select all the records where teh values of `cola` in `a` are present in the values of `cola` in `b`.

**Answer:** 
```{Postgres}
SELECT *
FROM a 
WHERE cola IN(
    SELECT DISTINCT cola
    FROM b
);
```

***

### Bonus 15 (Postgres)

**Topics**: Conditional Processing, Creating Dummy Variables, and Aggregating

***
#### Question 1 
***

Consider the following data 

```{Postgres}
DROP TABLE IF EXISTS a; 
CREATE TABLE a (cola int);
INSERT INTO a VALUES (null),(1),(5),(3),(null),(2);
SELECT * FROM a
```
If `col1` is null, return 0, otherwise return the value in `col1`. Write this in the shortest way possible.

**Answer:** `SELECT coalesce(col1,0) FROM a`

***    
#### **Question 2**
***

Consider the following data 

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a (id int, x text);
INSERT INTO a VALUES (0,'a'),(1,'b'),(2,'a'),(0,'b'),(1,'b'),(2,'a');
SELECT * FROM a;
```

Write a sql query that counts the number of `b` per id. *Do NOT* use a `WHERE` clause. Order the results by id.

**Answer:**

```{Postgres}
SELECT id,
    SUM(
        CASE WHEN x = 'a' THEN 0
             WHEN x = 'b' THEN 1
        END
    ) AS nbr_b
FROM a
GROUP BY id 
ORDER BY id;
```

***    
#### **Question 3**
***

Consider the following data

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a (col int);
INSERT into a VALUES (0),(1),(2),(0),(1),(2);
SELECT * FROM a;
```

Write a pl/pgSQL anonymous block that loops through the values of `col` in `a` and prints to the screen:

* $0$ when the `col` is $0$
* $1$ when the `col` is $1$
* $>1$ for all others 

**Answer:** 

```{Postgres}
DO $$
DECLARE
    v int;
BEGIN
    for v IN (SELECT col FROM a ORDER BY col) LOOP 
    IF v = 0 THEN raise notice '0';
    elsif v = 1 THEN raise notice '1';
    else raise notice '>1';
    END IF;
END LOOP;
END $$
```

***
#### Question 4
***

Consider the table below

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a(
    cat_variable1 text,
    cat_variable2 text);
INSERT INTO a (cat_variable1,cat_variable2)
VALUES ('ab cd e','e'),
       ('b','d'),
       ('ab cd e','e'),
       ('c','d');
SELECT * FROM a;
```

Automatically (using an anonymous) block create dummies for `cat_variable1`, kaing sure that spacces in the variable names are automatically replaced by underscores. Look up and use the replace function in postgres.

**Answer:**

```{Postgres}
DO $$
DECLARE 
    category text;
    category_sanitized text;
    query1 text;
    query2 text;
    query3 text;
BEGIN
    -- make a copy of `a` for two reasons:
        -- have a backup in case something goes wrong
        -- it is used in the for-clause and therefore 
        -- in an active query when we alter it
    DROP TABLE IF EXISTS dummies;
    CREATE TABLE dummies AS SELECT cat_variable1 FROM a;

    FOR category IN (SELECT DISTINCT cat_variable1 FROM a) LOOP 

        category_sanitized = replace(category, ' ','_');

        query1 = 'ALTER TABLE dummies ADD COLUMN cat_variable1_' || category_sanitized || 'int';
        execute query1;

        query2 = 'UPDATE dummies SET cat_variable1_' || category_sanitized || '=0';
        execute query2 

        query3 = 'UPDATE dummies SET cat_variable1_' || category_sanitized '=1 where replace(cat_variable1, '' '',''_'') = ''' || category_sanitized || '''';
        -- the four quotes at the end of query 3:
            -- the first quote starts the string
            -- second quote escapes the next one 
            -- third quote is the quote that gets added as follows: 'category'
            -- fourth quote ends the string 
        execute query3
    
    END LOOP;
END $$

SELECT * FROM dummies
```

**Part 2:** Expand your solution to part 1 to also make dummies automatically for `cat_variable2`

**Answer:**

```{Postgres}
DROP TABLE IF EXISTS a;
CREATE TABLE a(
    cat_variable1 text,
    cat_variable2 text);
INSERT INTO a (cat_variable1,cat_variable2)
VALUES ('ab cd e','e'),
       ('b','d'),
       ('ab cd e','e'),
       ('c','d');
SELECT * FROM a;

DO $$
DECLARE
    category record;
    category_sanitized text;
    query1 text;
    query2 text;
    query3 text;
BEGIN 

    DROP TABLE IF EXISTS dummies;
    CREATE TABLE dummies AS SELECT * FROM a;

FOR category IN (SELECT DISTINCT cat_variable1 AS VALUE, 'cat_variable1' AS variable FROM a
                UNION 
                SELECT DISTINCT cat_variable2, 'cat_variable2' FROM a) LOOP

    category_sanitized = replace(category.value, '','_');

    query1 = 'ALTER TABLE dummies ADD COLUMN ' || category.variable || '_' || category_sanitized || 'int';
    execute query1;

    query2 = 'UPDATE dummies SET ' || category.variable || '_' || category_sanitized || '=0';
    execute query2;

    query3 = 'UPDATE dummies SET ' || category.variable || '_' || category_sanitized || '=1
            where replace(' || category.variable || ', '' '',''_'') = ''' || category_sanitized || '''';
    execute query3;
    END LOOP;
END $$;

SELECT * FROM dummies;
```

***

### Bonus 16 (Postgres)

**Topics**: Working with Time

***
#### Question 1 
***

Write a query that subtracts one year and three days from today's date. The output columns should bec alled `today` and `one_year_and_three_days_ago`.

**Answer:**

```{Postgres}
SELECT current_date AS today,
       current_date - interval '1 year' - interval '3 days' AS one_year_and_three_days_ago;
```

***
#### Question 2
***

Consider the following three datasets 

```{Postgres, B16Q2}
DROP TABLE IF EXISTS a;
DROP TABLE IF EXISTS b;
DROP TABLE IF EXISTS c;

CREATE TABLE a (id int, col_a int);
CREATE TABLE b (id int, col_b int);
CREATE TABLE c (id int, col_c int);

INSERT INTO a VALUES (1,100),(2,200),(3,300);
INSERT INTO b VALUES (2,400),(3,500),(4,600);
INSERT INTO c VALUES (1,700),(3,800),(4,900);
```

In a single query *INNER JOIN* `a` and `b` and then *INNER JOIN* the joined `a_b` table and `c`. The join key is `id`. Write *two* different queries that accomplish that task: one with a multi-join and one with two single joins and a subquery

**Answer:**

```{Postgres}
SELECT *
FROM a 
INNER JOIN b ON a.id = b.id 
INNER JOIN c ON a.id = c.id; 

SELECT *
FROM(SELECT a.id, col_a, col_b
     FROM a
     INNER JOIN b ON a.id = b.id) AS a_b 
INNER JOIN c ON a_b.id = c.id;
```

***

### Bonus 17 (Postgres)

**Topics**: Working with Time

***
#### Question 1 
***

Create a function called `c` that returns the following information from the `information_schema.columns` table: `ordinal_position`, `column_name`, `data_type`. The input argument is a table name. Test the function on the subcriptions table. 

**Answer:**

```{Postgres}
-- We fist need to verify the column types 
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'columns'
AND column_name IN ('ordinal_position','column_name','data_type');

-- Next we can use these types in your create type statement

DROP TYPE coltype cascade;

CREATE TYPE coltype AS (ordinal_position integer, column_name name, data_type varchar);

DROP FUNCTION c;

CREATE OR REPLACE FUNCTION c (tab_name text) RETURNS setof coltype AS
$$
SELECT ordinal_position, column_name, data_type
FROM information_schema.columns
WHERE table_name = tab_name;
$$
language 'sql';

-- Test Function:
SELECT * FROM c('subscriptions');
```
***
#### Question 2
***

How long does it take to load the first row? Write the query to obtain that information. 

**Answer:** `explain analyze SELECT * FROM subscriptions;`

***
#### Question 3
***

* Step 1: Verify if table `a` is unlogged
* Step 2: Make table unlogged by altering it: https://www.postgresql.org/docs/current/sql-altertable.html
* Step 3: Verify if table `a` is now unlogged 
* Step 4: Make table `a` logged
* Step 5: Verify is table `a` is now logged 

```{Postgres}
SELECT relname FROM pg_class WHERE relpersistence = 'u';

ALTER TABLE a SET unlogged; 

SELECT relname FROM pg_class WHERE relpersistence = 'u';

ALTER TABLE a SET logged; 

SELECT relname FROM pg_class WHERE relpersistence = 'u';
```

***
#### Question 4
***

* Step 1: Explain and analyze the following query: 
`SELECT subcriptionid FROM subscriptions WHERE subcriptionid = '1013257';`
* Step 2: What are the first two words of the query plan, and what is th execution  time?
* Step 3: Create an index called `subscriptions_subscriptionid` on table `subscriptions` and column `subscriptionid`
* Step 4: Explain and analyze the query again (no need to copy and paste the statement from step 1) and write down the first three words of the query plan and what the execution time is

**Answer:**

```{Postgres}
-- Step 1
explain analyze SELECT subscriptionid FROM subcriptions WHERE subcriptionid='1013257';

-- Step 2:
-- Seq Scan (i.e., is scans all the instances)

-- Step 3: 
DROP index IF EXISTS subscriptions_subscriptionid;
CREATE index subscriptions_subscriptionid ON subscriptions (subscriptionid);

-- Step 4:
-- Index Only Scan
```

***

### Bonus 19 (PySpark)

***
#### Question 1 
***

Run the following code:

```{PySpark}
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[8]") \
    .appName("my_app") \
    .getOrCreate()

file_path = '/Users/mballin2/Desktop/subscriptions.csv'

from pyspark.sql.types import *

schema = StructType([
    StructField("SubscriptionID", StringType()),
    StructField("CustomerID", StringType()),
    StructField("ProductID", StringType()),
    StructField("Pattern", IntegerType()),
    StructField("StartDate", StringType()),
    StructField("EndDate", StringType()),
    StructField("NbrNewspapers", IntegerType()),
    StructField("NbrStart", IntegerType()),
    StructField("RenewalDate", StringType()),
    StructField("PaymentType", StringType()),
    StructField("PaymentStatus", StringType()),
    StructField("PaymentDate", StringType()),
    StructField("FormulaID", StringType()),
    StructField("GrossFormulaPrice", DoubleType()),
    StructField("NetFormulaPrice", DoubleType()),
    StructField("NetNewspaperPrice", DoubleType()),
    StructField("ProductDiscount", DoubleType()),
    StructField("FormulaDiscount", DoubleType()),
    StructField("TotalDiscount", DoubleType()),
    StructField("TotalPrice", DoubleType()),
    StructField("TotalCredit", DoubleType())
])

subscriptions = spark.read.csv(file_path, 
                               header = True, 
                               sep = ';',
                               schema = schema)
```

Subset the `subscriptions` data to *only* records where `SubscriptionID` equals $1001272$. Print those records to the screen using *two* different ways.

**Answer:**

```{PySpark}
# Way 1
subscriptions.filter(subscriptions.SubscriptionID == '1001272').show()

# Way 2
subscriptions.filter('SubscriptionID = ''1001272''').show()

# Way 3

subscriptions.createOrReplaceTempView('subscriptions')

spark.sql('select * from subscriptions where subscriptionID = ''1001272''').show()
```

***

### Bonus 20 (PyMongo)

***
#### Question 2 
***

Run the following code:

```{PyMongo}
from pymongo import MongoClient

# Client connect to localhost by default
client = MongoClient()

# List all the databases
client.list_database_names()

# Create an empty database called `peloton`
# And create handle called `db`
# If you need to drop database: client.drop_database('peloton')
db = client['peloton']

# Get data from Peloton, Inc. Twitter Account 
import requests 

response = requests.get('http://ballings.co/hidden/aCRM/data/tweets_peloton.json')
data = response.json()

# Create collection called `tweets` and insert data
db['tweets'].insert_many(data)
```

Write a filter to *only* return the documents that have the word `fitness` in the text field of the documents. Count the documents that pass that filter. Display the text of the documents that pass the filter, one by one, using a loop. 

**Answer:**

```{PyMongo}
filter = {'text': {'$regex': 'fitness'}}
nbr_docs = db.tweets.count_documents(filter)
doc_cursor = db.tweets.find(filter)
for i in range(nbr_docs):
    print(doc_cursor.next()['text'])
```


***
## 3 - Class Notes
***

***
### Class 21 - March 31, 2021
***

#### 2.7.2 - Read Data from Postgres Into Pyton

#### 3.4.3 - PostgreSQL: Conditional Processing

#### 3.5.3 - PostgreSQL: Timing Code

#### 3.7.3 - PostgreSQL: Creating Dummy Variables

#### 3.8.3 - PostgreSQL: Aggregating

***
### Class 22 - April 5, 2021
***

#### 3.9.3 - PostgreSQL: Merging

#### 3.11.3 - PostgreSQL: Working with Dates

***
### Class 23 - April 7, 2021
***

#### 3.13.3 - PostgreSQL: Creating Functions

#### 3.15.3 - PostgreSQL: Speeding Up Code

***
### Class 26 - April 19, 2021
***

#### 3.15.2.1 - Spark Components

#### 3.15.2.2 - Spark: How does it speed up data processing?

***
### Class 27 - April 21, 2021
***

#### 3.15.2.4 - Spark: Main Operators

****
## Class 28 - April 26, 2021
***

#### 4 - MongoDB: PyMongo